In [1]:
import pandas as pd
import psycopg2
import warnings
warnings.filterwarnings('ignore')
import numpy as np

In [2]:
user_data = pd.read_sql(
    """SELECT * FROM public.user_data""",
    con="postgresql://robot-secret"
        "postgres.lab.karpov.courses:secret"
)

user_data.head()

,user_id,gender,age,country,city,exp_group,os,source
0,200,1,34,Russia,Degtyarsk,3,Android,ads
1,201,0,37,Russia,Abakan,0,Android,ads
2,202,1,17,Russia,Smolensk,4,Android,ads
3,203,0,18,Russia,Moscow,1,iOS,ads
4,204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads


In [3]:
user_data.shape

(163205, 8)

In [4]:
user_data['user_id'].nunique()

163205

In [5]:
user_data.to_csv('user_data.csv')

In [6]:
post_df = pd.read_sql(
    """SELECT * FROM public.post_text_df""",
    con="postgresql://robot-secret"
        "postgres.lab.karpov.courses:secret"
)

post_df.head()

,post_id,text,topic
0,1,UK economy facing major risks\n\nThe UK manufa...,business
1,2,Aids and climate top Davos agenda\n\nClimate c...,business
2,3,Asian quake hits European shares\n\nShares in ...,business
3,4,India power shares jump on debut\n\nShares in ...,business
4,5,Lacroix label bought by US firm\n\nLuxury good...,business


In [7]:
post_df.to_csv('post_df.csv')

In [8]:
feed_data = pd.read_sql(
    """SELECT * FROM public.feed_data limit 100000""",
    con="postgresql://robot-secret"
        "postgres.lab.karpov.courses:secret"
)

feed_data.head()

,timestamp,user_id,post_id,action,target
0,2021-11-10 20:22:48,145195,4738,view,0
1,2021-11-13 12:45:48,145195,2835,view,1
2,2021-11-13 12:48:30,145195,2835,like,0
3,2021-11-13 12:48:32,145195,4374,view,0
4,2021-11-13 12:51:07,145195,5534,view,0


In [9]:
feed_data.to_csv('feed_data.csv')

In [10]:
df = pd.merge(
    feed_data,
    user_data,
    on='user_id',
    how='left'
)

df = pd.merge(
    df,
    post_df,
    on='post_id',
    how='left'
)

df.head()

,timestamp,user_id,post_id,action,target,gender,age,country,city,exp_group,os,source,text,topic
0,2021-11-10 20:22:48,145195,4738,view,0,0,21,Russia,Starokucherbayevo,0,iOS,organic,"As a fan of the old Doctor Who, and after the ...",movie
1,2021-11-13 12:45:48,145195,2835,view,1,0,21,Russia,Starokucherbayevo,0,iOS,organic,I think even those devastated by #LoanChargeSc...,covid
2,2021-11-13 12:48:30,145195,2835,like,0,0,21,Russia,Starokucherbayevo,0,iOS,organic,I think even those devastated by #LoanChargeSc...,covid
3,2021-11-13 12:48:32,145195,4374,view,0,0,21,Russia,Starokucherbayevo,0,iOS,organic,This TVM seems to have polarised opinions amon...,movie
4,2021-11-13 12:51:07,145195,5534,view,0,0,21,Russia,Starokucherbayevo,0,iOS,organic,I havent laughed this much in a long time - or...,movie


In [11]:
df.isna().sum()

timestamp    0
user_id      0
post_id      0
action       0
target       0
gender       0
age          0
country      0
city         0
exp_group    0
os           0
source       0
text         0
topic        0
dtype: int64

In [12]:
likes = df.query('action=="like"').groupby(['user_id', 'post_id'], as_index=False)['target'].agg('count')\
        .rename(columns={'target':'like'})

In [13]:
likes.shape

(10040, 3)

In [14]:
views = df.query('action=="view"').groupby(['user_id', 'post_id'], as_index=False)['target'].agg('sum')\
        .rename(columns={'target':'lliked_view'})

In [15]:
liked_views = views.merge(likes, on=['user_id', 'post_id'])
liked_views.shape

(10040, 4)

In [16]:
liked_views.head()

,user_id,post_id,lliked_view,like
0,29863,548,1,1
1,29863,1083,1,1
2,29863,1581,1,1
3,29863,4935,1,1
4,29863,5694,1,1


In [17]:
liked_views.lliked_view.value_counts()

1    9962
2      76
3       2
Name: lliked_view, dtype: int64

Поскольку у всех просмотров в данной таблице target>0, то они все лайкнуты сразу после просмотра. 
Также их количество примерно равно количеству постов с like. Однако, из таблицы куда-то потерялись 2 поста с like - возможно, данные по их просмотру "срезались" при выгрузке или при сплитовании на трейн и тест. Таким образом, можно удалить из таблицы все посты с like, а по оставшимся постам с view посчитать среднее по таргету - так мы получим метрики, как часто пользователь ставит лайки и как часто ставят лайки этому посту.

In [18]:
df = df.query('action=="view"')

In [19]:
user_likes = df.groupby('user_id', as_index=False).target.mean()\
            .rename(columns={'target':'user_rate'}) #соотношение лайков и просмотров для каждого пользователя
post_rate = df.groupby('post_id', as_index=False).target.mean()\
            .rename(columns={'target':'post_rate'}) #"рейтинг" каждого поста

In [20]:
user_likes = df.groupby('user_id')['target'].mean() #соотношение лайков и просмотров для каждого пользователя
post_rate = df.groupby('post_id')['target'].mean()  #"рейтинг" каждого поста

In [21]:
# создадим колонки с рейтингом
df['user_rate'] = df['user_id'].map(user_likes)
df['post_rate'] = df['post_id'].map(post_rate)

Поскольку один пользователь может просматривать пост несколько раз, необходимо оставить только по одному просмотру на пост. Если пост лайкнут - значит оставить этот просмотр, если не лайнкут - оставить один из просмотров.

In [38]:
# сразу создадим дф с признаками пользователей
user_features = user_data.merge(df[['user_id', 'user_rate']].drop_duplicates(), on='user_id', how='left')\
                .fillna(df['user_rate'].mean())

In [40]:
# для каждого поста оставим только минимальное время, которое будем считать временем публикации

post_time = feed_data.groupby('post_id', as_index=False)['timestamp'].min()

# сразу создадим дф с признаками постов
post_features = post_df.merge(df[['post_id', 'post_rate']].drop_duplicates(), on='post_id', how='left')\
                .fillna(df['post_rate'].mean())

post_features = post_features.merge(post_time, on='post_id', how='left')

In [23]:
# создаём датафрейм, в котором будут только лайкнутые посты
liked = df.query('target==1').groupby(['user_id', 'post_id'], as_index=False).size()
liked.shape

(10041, 3)

In [24]:
# создаём датафрейм с просмотрами без лайков
non_liked = df.query('target==0').groupby(['user_id', 'post_id'], as_index=False).size()

# смотрим, сколько раз один и тот же пост может попадаться одному и тому же пользователю
non_liked['size'].unique()

array([1, 2, 3, 4], dtype=int64)

In [25]:
# далее для каждой пары юзер-пост проверяем, есть ли этот пост среди лайков.
# если нет - ставим 0, если есть - 1. Далее отфильтровываем 1.
for user in non_liked['user_id'].unique():
    liked_posts = liked[liked['user_id']==user]['post_id'].to_list()
    for post in non_liked[non_liked['user_id']==user]['post_id'].unique():
        if post not in liked_posts:
            non_liked.loc[(non_liked['user_id']==user)&(non_liked['post_id']==post), 'size'] = 0
        else:
            non_liked.loc[(non_liked['user_id']==user)&(non_liked['post_id']==post), 'size'] = 1

In [26]:
non_liked['size'].unique()

array([0, 1], dtype=int64)

In [27]:
non_liked = non_liked.query('size==0') # оставим только посты без лайков
non_liked.shape

(75686, 3)

In [28]:
user_posts = pd.concat([non_liked, liked]).rename(columns={'size':'target'}) # объединим всё в единый дф
user_posts.head()

,user_id,post_id,target
0,29863,1269,0
1,29863,1297,0
2,29863,1524,0
3,29863,1601,0
4,29863,1806,0


In [41]:
user_features = user_posts.merge(user_feat, on='user_id', how='left')
user_features = user_features.merge(post_features, on='post_id', how='left')
user_features.head()

,user_id,post_id,target,gender,age,country,city,exp_group,os,source,user_rate,text,topic,post_rate,timestamp
0,29863,1269,0,0,19,Russia,Tambov,1,iOS,ads,0.241379,Blair and Brown criticised by MPs\n\nLabour MP...,politics,0.103448,2021-10-01 21:54:51
1,29863,1297,0,0,19,Russia,Tambov,1,iOS,ads,0.241379,Could rivalry overshadow election?\n\nTony Bla...,politics,0.095238,2021-10-02 14:07:02
2,29863,1524,0,0,19,Russia,Tambov,1,iOS,ads,0.241379,Italy aim to rattle England\n\nItaly coach Joh...,sport,0.142857,2021-11-17 12:30:06
3,29863,1601,0,0,19,Russia,Tambov,1,iOS,ads,0.241379,Isinbayeva claims new world best\n\nPole vault...,sport,0.000000,2021-10-01 20:27:04
4,29863,1806,0,0,19,Russia,Tambov,1,iOS,ads,0.241379,Campbell rescues Arsenal\n\nSol Campbell prove...,sport,0.157895,2021-10-02 11:03:11


In [42]:
user_features.shape

(85727, 15)

In [43]:
user_features.to_csv('user_features.csv')

In [202]:
### Разделим на трейн-тест
from sklearn.utils import shuffle
user_features = shuffle(user_features)

train = user_features.iloc[:-20000].copy()
test = user_features.iloc[-20000:].copy()

In [203]:
train_new = train.drop(['user_id', 'post_id',  
                         'exp_group', 'text', 'timestamp'], axis=1) 

test_new = test.drop(['user_id','post_id',  
                        'exp_group', 'text', 'timestamp'], axis=1) 

# удаляем лишние колонки. exp_group будет использована в АВ тестировании, а text - в NLP.

In [204]:
train_new.head()

,target,gender,age,country,city,os,source,user_rate,topic,post_rate
24210,0,1,29,Russia,Lipetsk,Android,ads,0.042208,politics,0.086957
54656,0,1,27,Ukraine,Kamianske,iOS,organic,0.234513,politics,0.151515
84051,1,0,27,Russia,Volzhsk,Android,organic,0.056686,tech,0.200000
54420,0,1,30,Russia,Roslavl,Android,organic,0.078049,politics,0.157895
57282,0,0,38,Russia,Moscow,iOS,organic,0.094241,business,0.055556


In [205]:
X_train = train_new.drop('target', axis=1)
X_test = test_new.drop('target', axis=1)

y_train = train['target']
y_test = test['target']

In [206]:
cat_features=['gender', 'country', 'city', 'os', 'source', 'topic']

In [207]:
from catboost import CatBoostClassifier

catboost = CatBoostClassifier()


catboost.fit(X_train,
             y_train,
             cat_features=cat_features,
             )

Learning rate set to 0.097924
0:	learn: 1.1848808	total: 50ms	remaining: 50s
1:	learn: 1.0405930	total: 97.3ms	remaining: 48.5s
2:	learn: 0.9333803	total: 123ms	remaining: 41s
3:	learn: 0.8486286	total: 148ms	remaining: 36.9s
4:	learn: 0.7807091	total: 187ms	remaining: 37.3s
5:	learn: 0.7237009	total: 212ms	remaining: 35.2s
6:	learn: 0.6739017	total: 276ms	remaining: 39.1s
7:	learn: 0.6349867	total: 289ms	remaining: 35.9s
8:	learn: 0.5992861	total: 329ms	remaining: 36.3s
9:	learn: 0.5671989	total: 394ms	remaining: 39s
10:	learn: 0.5394359	total: 443ms	remaining: 39.8s
11:	learn: 0.5153767	total: 509ms	remaining: 41.9s
12:	learn: 0.4949813	total: 541ms	remaining: 41.1s
13:	learn: 0.4760438	total: 593ms	remaining: 41.8s
14:	learn: 0.4593731	total: 662ms	remaining: 43.4s
15:	learn: 0.4448357	total: 695ms	remaining: 42.8s
16:	learn: 0.4319014	total: 766ms	remaining: 44.3s
17:	learn: 0.4206219	total: 834ms	remaining: 45.5s
18:	learn: 0.4101528	total: 907ms	remaining: 46.8s
19:	learn: 0.4010

162:	learn: 0.3091990	total: 11s	remaining: 56.4s
163:	learn: 0.3091843	total: 11.1s	remaining: 56.4s
164:	learn: 0.3091750	total: 11.1s	remaining: 56.4s
165:	learn: 0.3090878	total: 11.2s	remaining: 56.4s
166:	learn: 0.3090445	total: 11.3s	remaining: 56.4s
167:	learn: 0.3089856	total: 11.4s	remaining: 56.3s
168:	learn: 0.3089131	total: 11.4s	remaining: 56.3s
169:	learn: 0.3088781	total: 11.5s	remaining: 56.4s
170:	learn: 0.3088296	total: 11.7s	remaining: 56.5s
171:	learn: 0.3087414	total: 11.7s	remaining: 56.5s
172:	learn: 0.3087198	total: 11.8s	remaining: 56.5s
173:	learn: 0.3086584	total: 11.9s	remaining: 56.4s
174:	learn: 0.3086279	total: 11.9s	remaining: 56.3s
175:	learn: 0.3085981	total: 12s	remaining: 56.3s
176:	learn: 0.3085561	total: 12.1s	remaining: 56.3s
177:	learn: 0.3085075	total: 12.2s	remaining: 56.3s
178:	learn: 0.3084886	total: 12.3s	remaining: 56.3s
179:	learn: 0.3084060	total: 12.4s	remaining: 56.3s
180:	learn: 0.3083777	total: 12.4s	remaining: 56.3s
181:	learn: 0.30

323:	learn: 0.3039212	total: 23.4s	remaining: 48.9s
324:	learn: 0.3038883	total: 23.5s	remaining: 48.8s
325:	learn: 0.3038570	total: 23.6s	remaining: 48.8s
326:	learn: 0.3038062	total: 23.7s	remaining: 48.7s
327:	learn: 0.3037648	total: 23.7s	remaining: 48.7s
328:	learn: 0.3037352	total: 23.9s	remaining: 48.7s
329:	learn: 0.3036899	total: 23.9s	remaining: 48.6s
330:	learn: 0.3036771	total: 24s	remaining: 48.5s
331:	learn: 0.3036686	total: 24.1s	remaining: 48.5s
332:	learn: 0.3036633	total: 24.2s	remaining: 48.4s
333:	learn: 0.3036452	total: 24.2s	remaining: 48.3s
334:	learn: 0.3036273	total: 24.3s	remaining: 48.3s
335:	learn: 0.3035915	total: 24.4s	remaining: 48.3s
336:	learn: 0.3035286	total: 24.5s	remaining: 48.2s
337:	learn: 0.3035010	total: 24.6s	remaining: 48.2s
338:	learn: 0.3034781	total: 24.7s	remaining: 48.1s
339:	learn: 0.3034432	total: 24.8s	remaining: 48s
340:	learn: 0.3034126	total: 24.8s	remaining: 48s
341:	learn: 0.3033774	total: 24.9s	remaining: 47.9s
342:	learn: 0.3033

482:	learn: 0.2995370	total: 35.5s	remaining: 38s
483:	learn: 0.2994632	total: 35.6s	remaining: 37.9s
484:	learn: 0.2994557	total: 35.6s	remaining: 37.8s
485:	learn: 0.2994291	total: 35.7s	remaining: 37.8s
486:	learn: 0.2994095	total: 35.8s	remaining: 37.7s
487:	learn: 0.2993609	total: 35.9s	remaining: 37.6s
488:	learn: 0.2993324	total: 35.9s	remaining: 37.5s
489:	learn: 0.2993163	total: 36s	remaining: 37.5s
490:	learn: 0.2992636	total: 36.1s	remaining: 37.4s
491:	learn: 0.2992438	total: 36.1s	remaining: 37.3s
492:	learn: 0.2992209	total: 36.2s	remaining: 37.2s
493:	learn: 0.2992036	total: 36.3s	remaining: 37.2s
494:	learn: 0.2991930	total: 36.4s	remaining: 37.1s
495:	learn: 0.2991755	total: 36.4s	remaining: 37s
496:	learn: 0.2991412	total: 36.5s	remaining: 37s
497:	learn: 0.2991358	total: 36.6s	remaining: 36.9s
498:	learn: 0.2991078	total: 36.7s	remaining: 36.8s
499:	learn: 0.2990916	total: 36.8s	remaining: 36.8s
500:	learn: 0.2990751	total: 36.8s	remaining: 36.7s
501:	learn: 0.299059

641:	learn: 0.2953849	total: 47.8s	remaining: 26.7s
642:	learn: 0.2953707	total: 47.9s	remaining: 26.6s
643:	learn: 0.2953499	total: 48s	remaining: 26.5s
644:	learn: 0.2953405	total: 48.1s	remaining: 26.5s
645:	learn: 0.2953185	total: 48.2s	remaining: 26.4s
646:	learn: 0.2952933	total: 48.3s	remaining: 26.3s
647:	learn: 0.2952620	total: 48.4s	remaining: 26.3s
648:	learn: 0.2952312	total: 48.4s	remaining: 26.2s
649:	learn: 0.2951982	total: 48.5s	remaining: 26.1s
650:	learn: 0.2951443	total: 48.6s	remaining: 26.1s
651:	learn: 0.2951288	total: 48.7s	remaining: 26s
652:	learn: 0.2950744	total: 48.8s	remaining: 25.9s
653:	learn: 0.2950511	total: 48.8s	remaining: 25.8s
654:	learn: 0.2950365	total: 48.9s	remaining: 25.8s
655:	learn: 0.2950262	total: 49s	remaining: 25.7s
656:	learn: 0.2949929	total: 49.1s	remaining: 25.6s
657:	learn: 0.2949734	total: 49.2s	remaining: 25.6s
658:	learn: 0.2949599	total: 49.3s	remaining: 25.5s
659:	learn: 0.2949409	total: 49.4s	remaining: 25.4s
660:	learn: 0.2949

803:	learn: 0.2919375	total: 1m 2s	remaining: 15.1s
804:	learn: 0.2919323	total: 1m 2s	remaining: 15s
805:	learn: 0.2919054	total: 1m 2s	remaining: 15s
806:	learn: 0.2918708	total: 1m 2s	remaining: 14.9s
807:	learn: 0.2918434	total: 1m 2s	remaining: 14.8s
808:	learn: 0.2918062	total: 1m 2s	remaining: 14.7s
809:	learn: 0.2917898	total: 1m 2s	remaining: 14.7s
810:	learn: 0.2917629	total: 1m 2s	remaining: 14.6s
811:	learn: 0.2917464	total: 1m 2s	remaining: 14.5s
812:	learn: 0.2917113	total: 1m 2s	remaining: 14.4s
813:	learn: 0.2916691	total: 1m 2s	remaining: 14.4s
814:	learn: 0.2916440	total: 1m 2s	remaining: 14.3s
815:	learn: 0.2916359	total: 1m 2s	remaining: 14.2s
816:	learn: 0.2916256	total: 1m 3s	remaining: 14.1s
817:	learn: 0.2916172	total: 1m 3s	remaining: 14.1s
818:	learn: 0.2915934	total: 1m 3s	remaining: 14s
819:	learn: 0.2915708	total: 1m 3s	remaining: 13.9s
820:	learn: 0.2915431	total: 1m 3s	remaining: 13.8s
821:	learn: 0.2915338	total: 1m 3s	remaining: 13.7s
822:	learn: 0.2914

961:	learn: 0.2886501	total: 1m 14s	remaining: 2.94s
962:	learn: 0.2886271	total: 1m 14s	remaining: 2.86s
963:	learn: 0.2886028	total: 1m 14s	remaining: 2.79s
964:	learn: 0.2885947	total: 1m 14s	remaining: 2.71s
965:	learn: 0.2885630	total: 1m 14s	remaining: 2.63s
966:	learn: 0.2885552	total: 1m 14s	remaining: 2.55s
967:	learn: 0.2885389	total: 1m 14s	remaining: 2.48s
968:	learn: 0.2885222	total: 1m 14s	remaining: 2.4s
969:	learn: 0.2885001	total: 1m 15s	remaining: 2.32s
970:	learn: 0.2884920	total: 1m 15s	remaining: 2.24s
971:	learn: 0.2884856	total: 1m 15s	remaining: 2.17s
972:	learn: 0.2884742	total: 1m 15s	remaining: 2.09s
973:	learn: 0.2884574	total: 1m 15s	remaining: 2.01s
974:	learn: 0.2884420	total: 1m 15s	remaining: 1.93s
975:	learn: 0.2884218	total: 1m 15s	remaining: 1.86s
976:	learn: 0.2884052	total: 1m 15s	remaining: 1.78s
977:	learn: 0.2883780	total: 1m 15s	remaining: 1.7s
978:	learn: 0.2883445	total: 1m 15s	remaining: 1.62s
979:	learn: 0.2883121	total: 1m 15s	remaining: 1

In [208]:
test_new = df.iloc[-20000:].copy()

X_test['prob'] = catboost.predict_proba(X_test)[:, 1]
X_test['pred'] = catboost.predict(X_test)
X_test['target'] = y_test
X_test['user_id'] = test_new['user_id']
X_test['post_id'] = test_new['post_id']

In [209]:
hitrate5 = []

for user in X_test['user_id'].unique():
    part = X_test[X_test['user_id']==user]
    if sum(part['pred']==1)==0:
        continue # пропускаем пользователя, если для него нет рекомендаций
    elif sum(part[part['pred']==1].sort_values(by='prob', ascending=False)[:5]['target']==1)>0:
        hitrate5.append(1)
    else:
        hitrate5.append(0)
        
print(f"Среднее HitRate@5 по пользователям из теста: {np.mean(hitrate5)}")

Среднее HitRate@5 по пользователям из теста: 1.0


In [210]:
catboost.save_model('catboost_model',
                           format="cbm")